# Установка

In [ ]:
# @title Установка пакетов
from IPython.display import clear_output

!pip install --upgrade tiktoken
!pip install langchain openai
!pip install faiss-cpu

clear_output()

import getpass
import openai
import os
def get_key_ОpenAI():
  openai.api_key = getpass.getpass(prompt='Введите секретный ключ для сервиса chatGPT: ')
  os.environ["OPENAI_API_KEY"] = openai.api_key

get_key_ОpenAI()

# Для работы с Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
#@title Импорт библиотек и Сервисные функции
import gdown
from IPython.display import clear_output
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile

import time
import os
import openai
import tiktoken
import re
import os
import json
from tqdm.auto import tqdm
import getpass


# ----------------------------------
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"
MODEL_GPT4 = "gpt-4-0613"
# ----------------------------------

clear_output()

class WorkerОpenAI():
  def __init__(self, \
               system_promt = " ", \
               system_promt_lector = " ", \
               mod = MODEL_TURBO_16K, \
               content_topics = None, \
               save_project = '/content/'):
    self.model = mod
    self.save_project  = save_project

    if content_topics:
      self.content_topics = self.load_txt_file(content_topics)

    # системные настройки
    self.system_promt = self.load_document_text(system_promt)
    self.speaker_system_promt = self.load_document_text(system_promt_lector)


  def load_document_text(self, url: str) -> str:
      # функция для загрузки документа по ссылке из гугл док
      match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
      if match_ is None:
          raise ValueError('Invalid Google Docs URL')
      doc_id = match_.group(1)
      response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
      response.raise_for_status()
      text = response.text
      return text


  def load_txt_file(self, file_path):
      with open(file_path, 'r') as file_:
          text = file_.read()
      return text

  # пример подсчета токенов
  def num_tokens_from_messages(self, messages):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(self.model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      # if self.model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      if self.model in ["gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-4-0613"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start>assistant
          return num_tokens
      else:
          raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {self.model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


  def create_embedding_faiss_db(self, doc_txt_dir="/content/", \
                                faiss_db_dir ="/content/", \
                                start_idx = 0, \
                                collection_name = " "):

    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    # Для Копирайтера
    self.splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=1024, chunk_overlap=300)
    idx_file_folder = start_idx-1
    chunkID = idx_file_folder

    count_tokens = 0
    # проходимся по всем данным
    for _, file_ in enumerate(sorted(os.listdir(doc_txt_dir))):
        print("Загружается файл: ", file_)
        self.file_name = file_
        idx_file_folder +=1
        source_chunks = []
        # разбиваем на несколько частей с помощью метода split_text
        with open(doc_txt_dir + file_, "r") as f:
          for chunk in self.splitter.split_text(f.read()):
              chunkID += 1
              source_chunks.append(Document(page_content=chunk, \
                                  metadata={'source': file_,
                                            'chunkID': chunkID,
                                            "collection_name": collection_name,
                                            'idx_file_folder': idx_file_folder}))


        # Создание индексов документа и СОХРАНЕНИЕ
        # Если документ не пуст, то создать и сохранить базу индексов эмбеддингов отрезков документа
        if len(source_chunks) > 0:
            self.db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
            count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
            count_tokens += count_token
            print('Количество токенов в документе :', count_token)
            # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

            self.db.save_local(os.path.join(faiss_db_dir, collection_name, f"{str(idx_file_folder)}_db_initial__{file_[:20]}"))

    print('\nЦЕНА запроса создания базы индексов:', 0.0004 * (count_tokens / 1000), ' $')

# ДЛЯ ОДНОГО КОНКРЕТНОГО ФАЙЛА
  def create_embedding_one_file(self, doc_txt_dir="/content/", \
                                file_ = "Имя файла.txt" , \
                                faiss_db_dir ="/content/"):

    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    # Для Копирайтера
    self.splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=1024, chunk_overlap=0)
    chunkID = 0
    count_tokens = 0
    self.file_name = file_[:-3]
    print("Загружается файл: ", file_)
    # проходимся по всем данным
    source_chunks = []
    # разбиваем на несколько частей с помощью метода split_text
    with open(os.path.join(doc_txt_dir, file_), "r") as f:
      for chunk in self.splitter.split_text(f.read()):
          chunkID += 1
          source_chunks.append(Document(page_content=chunk, \
                              metadata={'source': file_,
                                        'chunkID': chunkID}))

    # Создание индексов документа и СОХРАНЕНИЕ
    # Если документ не пуст, то создать и сохранить базу индексов эмбеддингов отрезков документа
    if len(source_chunks) > 0:
        self.db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
        count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
        count_tokens += count_token
        print('Количество токенов в документе :', count_token)
        # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

        self.db.save_local(os.path.join(faiss_db_dir, f"db_initial__{self.file_name[:35]}"))

    print('\nЦЕНА запроса создания базы индексов:', 0.0004 * (count_tokens / 1000), ' $')




# ЗАПРОС в ChatGPT
  def get_ChatCompletion(self, model,  # указываем модель
                         messages,     # словарь запроса
                         temp=0.2):    # температуру

      completion = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temp
        )

      # print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
      # print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
      # print('===========================================: \n')
      return completion.choices[0].message.content

# ЗАПРОС на создание темы и подтемы по Материалам
  def get_search_materials_topics_subtopics(self, materials, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй отрывок материала c семинара: {materials}.

Выдели основные темы.
Дай компактный, сжатый, обобщенный список тем и подтем.
Темы Необходимо оформить _#, а подтемы оформить ##_.
Используй только такой пример, ничего не добавляй лишнего.
Пример составления списка:
_#...
##_...
_#...
##_...
        """}
        ]

    # example token count from the function defined above
    # print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")


# ПОИСК ТЕМЫ и ПОДТЕМЫ
  def search_topics_subtopics(self, num_chunk = 14):
    self.content_topics = ''
    materials = ""
    # Выбираем блоки ПОДРЯД
    len_chunk = len(self.db.docstore._dict)
    if len_chunk < num_chunk + 1:
        for _, doc in tqdm(self.db.docstore._dict.items()):
            materials += f"{doc.page_content}\n"
        self.get_search_materials_topics_subtopics(materials)
    else:
        for ind, (key, doc) in tqdm(enumerate(self.db.docstore._dict.items())):
          materials += f"{doc.page_content}\n"
          if (ind+1) % num_chunk == 0:
            self.get_search_materials_topics_subtopics(materials)
            materials = ""
    if materials != "":
        self.get_search_materials_topics_subtopics(materials)

    print('Собрали список тем и подтем: ')
    print(self.content_topics)

    with open(f'{self.save_project}_{self.file_name[:20]}__Topic_Subtopic.txt', "w") as f:
      f.write(self.content_topics)


# ОБЪЕДИНЯЕМ схожие ТЕМЫ
  def get_merge_topics(self, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй Темы и подтемы Семинара: {self.content_topics}.

Необходимо объединить похожие по смыслу темы или подтемы, записать компактно.
При необходимости перефразировать тему или подтему. Дай корректный список.
Темы оформи _#, а подтемы оформи ##_.
Используй только такой пример, ничего не добавляй лишнего.
Пример составления списка:
_#...
##_...
_#...
##_...
"""}
]
    # example token count from the function defined above
    # print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics = self.get_ChatCompletion(model_topics, messages)
      print('Итоговый список тем и подтем: ')
      print(self.content_topics)
      with open(f'{self.save_project}_{self.file_name[:20]}__Topic_Subtopic_Final.txt', "w") as f:
        f.write(self.content_topics)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

# ОРГАНИЗУЕМ текст блоками
  def organize_text(self, topic, num_chunks, model_topics):

    # Выборка документов по схожести с подтемой
    docs = self.db.similarity_search(topic, k = num_chunks)
    message_content = '\n'.join([doc.page_content + '\n' for i, doc in enumerate(docs)])

    messages = [
        {"role": "system", "content": f"{self.speaker_system_promt}"},
        {"role": "user", "content": f"""Вот отрывки Вашей лекции:\n{message_content}

Опираясь только на информацию с Лекции, указанной выше, расскажите подробнее по Теме: {topic}.
"""}
      ]

    # example token count from the function defined above
    # print('\n ===========================================: ')
    # print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.final_text += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

# ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ
  def organize_final_text(self, num_chunks = 8,
                          model_topics = "gpt-3.5-turbo-16k",
                          name = None,
                          db_path = None):
    if name:
      self.file_name = name

    if db_path:
      self.db = FAISS.load_local(db_path, OpenAIEmbeddings())

    # if db_path:
    #   for curr_base in os.listdir(db_path):
    #       self.db = FAISS.load_local(os.path.join(db_path, curr_base), OpenAIEmbeddings())

    self.final_text = ""
    list_topics = self.content_topics.split('\n')
    # проходимся по списку тем и подтем
    for ind, topic in tqdm(enumerate(list_topics)):
        if ind%5 == 0:
          with open(f'{self.save_project}{self.file_name[:20]}__loop{ind}.txt', "w") as f:
              f.write(self.final_text)
        # тему просто записываем в итоговый текст
        if ("_#" in topic) and not("_#" in list_topics[ind+1]):
          self.final_text += f'\n{topic}\n'
        # подтему передаем в ChatGPT
        else:
          self.final_text += f'\n{topic}\n'
          self.organize_text(topic, num_chunks, model_topics)

    with open(f'{self.save_project}{self.file_name[:20]}__final_text.txt', "w") as f:
      f.write(self.final_text)
    print("\nСоздали финальный документ для Базы Знаний (final_text)")





clear_output()




# Copywriter. Нейро-Копийтер для MarkdownHeaderTextSplitter

### Создаем текстовый файл с диалогами из Гугл таблицы

In [ ]:
import pandas as pd
import requests
from io import BytesIO

# функция для загрузки таблицы по ссылке из гугл драйв
def load_xls_pd(url: str, sheet_name: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Sheets URL')
    doc_id = match_.group(1)

    # Download the table as pandas
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=xlsx')
    response.raise_for_status()     #проверяет статус код ответа. Если получен ответ с кодом ошибки (4xx или 5xx), вызывается исключение HTTPError.
    data = pd.read_excel(BytesIO(response.content), sheet_name=0)
    #data = pd.read_excel(BytesIO(response.content),sheet_name='Sheet1') #чтение конкретного листа из Книги Excel

    '''
        Когда мы хотим прочитать данные Excel с помощью функции pd.read_excel(), она требует передачи ей пути к файлу или объекта, представляющего файл.
        В параметре io функции pd.read_excel() необходимо указать путь к файлу (в виде строки, содержащей путь к файлу) или объект файлового типа (такой как BufferedWriter, BufferedReader и другие).
        response.content возвращает содержимое ответа на запрос HTTP в виде байтового массива (bytes array).
        Чтобы передать эти данные в функцию pd.read_excel(), нужно создать объект файла из байтового массива.
        Для этой цели используется объект BytesIO из модуля io, который предоставляет интерфейс для работы с данными в памяти, как если бы они находились в файле.
    '''
    return data

# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = "https://docs.google.com/spreadsheets/d/165TBS6Fobxzx6rew29tzIDDe_4GuUDbRfUOpWNhew2E/edit?usp=sharing"
#sheet_name='Sheet1'
data = load_xls_pd(google_sheet_url, sheet_name)

# Шаг 2: Извлечь столбец "text"
text_column = data["text"]

# Шаг 3: Заменить "operatorMessage: Здравствуйте" на "<operatorMessage: Здравствуйте>"
text_column = text_column.str.replace("operatorMessage: Здравствуйте", "<operatorMessage: Здравствуйте>")

# Шаг 4: Сохранить в файл "Dialogs__.txt" в своей папке
with open("/content/drive/MyDrive/Stazhirovka_Kia/Dialogs/Dialogs_Unable_to_determine/BASE/kia_dialog_Unable_to_determine.txt", "w", encoding="utf-8") as f:
    for text in text_column:
        f.write(str(text) + "\n")

print("Файл kia_dialog_____________.txt успешно сохранен!")

In [ ]:
#@title Создаем объект для работы Copywriter
projects_dir = '/content/drive/MyDrive/Stazhirovka_Kia/Dialogs/Dialogs_Unable_to_determine'

Promt_copywriter = "https://docs.google.com/document/d/1zePK0OWfOrBma99zfxxU6QzG2mu866vGpK4IwyNzcEw/edit?usp=sharing"
Promt_lector = "https://docs.google.com/document/d/1eXZ27iOTuHtjtE4o2fWQlysZUudQ9-8rYi0eCGCupfM/edit?usp=sharing"

# если темы созданы
# topics_final = "/content/drive/MyDrive/Stazhirovka_Kia/Dialogs/___.txt"
# # Создаем объект для дообучения chatGPT
curator = WorkerОpenAI(system_promt = Promt_copywriter, # системный промт
                       system_promt_lector = Promt_lector, # промт Консультанта
                      #  content_topics = topics_final,
                       save_project = projects_dir)     # путь для сохранения готовых файлов

# Формируем базу по файлу txt
# # путь к материалам
doc_txt_dir = projects_dir + '/BASE/'
file_name = 'kia_dialog_Unable_to_determine.txt'



In [ ]:
# curator.create_embedding_faiss_db(doc_txt_dir = doc_txt_dir,   # путь к материалам
#                                 faiss_db_dir = db_initial,     # путь для сохранения исходной базы
#                                 start_idx = 0,                 # номер документа в базе
#                                 collection_name = 'УИИ_db_initial')  # наименование коллекции


curator.create_embedding_one_file(doc_txt_dir = doc_txt_dir,   # путь к материалам
                                  file_ = file_name,            # какой файл берем
                                  faiss_db_dir = projects_dir)     # путь для сохранения исходной базы

In [ ]:
# Смотрим созданные чанки
print(f"Текст разбит на чанки. Всего: {len(curator.db.docstore._dict.values())} шт.\n")
curator.db.docstore._dict.values()

In [ ]:
#@title Поиск темы и подтемы выборкой по 5 - 15 чанков

# Подаем в цикле и просим составить список тем и подтем.
curator.search_topics_subtopics()

In [ ]:
#@title Корректируем список тем и подтем (обобщаем, убираем дубли).
# Подаем сформированный Список тем и просим объединить похожие по смыслу темы или подтемы, записать компактно.
# При необходимости перефразировать тему или подтему.
curator.get_merge_topics()

## ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ

Выборка блоков документов по схожести с подтемой. Передаем собранные отрывки с Лекции и просим рассказать подробнее, опираясь на Лекцию.

In [ ]:
projects_dir = '/content/drive/MyDrive/Stazhirovka_Kia/Dialogs/Dialogs_Unable_to_determine'

Promt_copywriter = "https://docs.google.com/document/d/1zePK0OWfOrBma99zfxxU6QzG2mu866vGpK4IwyNzcEw/edit?usp=sharing"
Promt_lector = "https://docs.google.com/document/d/1zePK0OWfOrBma99zfxxU6QzG2mu866vGpK4IwyNzcEw/edit?usp=sharing"

# если темы созданы
topics_final = "/content/drive/MyDrive/Stazhirovka_Kia/Dialogs/Dialogs_Unable_to_determine/Dialogs_Unable_to_determinekia_dialog_Unable_to__темы_подтемы_итог.txt"
# # Создаем объект для дообучения chatGPT
curator = WorkerОpenAI(system_promt = Promt_copywriter, # системный промт
                       system_promt_lector = Promt_lector, # промт Лектора
                       content_topics = topics_final,
                       save_project = projects_dir)     # путь для сохранения готовых файлов


In [ ]:
# запускаем сборку Итогового Текста
name_file = 'БЗ_kia_Dialogs_Unable_to_determine'
db_initial = '/content/drive/MyDrive/Stazhirovka_Kia/Dialogs/Dialogs_Unable_to_determine/db_initial__kia_dialog_Unable_to_determine.'

curator.organize_final_text(name = name_file,
                            db_path = db_initial)

# curator.organize_final_text()

In [ ]:
# сформирован итоговый текст
curator.final_text

# MarkdownHeaderTextSplitter

In [ ]:
#@title функции
chat_manager_system = """
Ты программист, специалист по Нейронным сетям, Профессионал Data Science и Преподаватель в Университете Искусственного Интеллекта.
Ты Спикер на Вебинаре.
Тебе предоставят отрывок с Вебинара и попросят раскрыть подробнее одну из тем.

Твоя цель: Опираясь только на материал с Вебинара, подробно, развернуто рассказать по интересующей теме на русском языке.
"""

def get_chatgpt_answer(topic,  db, model = MODEL_TURBO_0613):
  # Выборка документов по схожести с вопросом
  docs = db.similarity_search(topic, k=4)
  message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n  ' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
  # print('message_content :\n ======================================== \n', self.message_content)

  messages = [
    {"role": "system", "content": f"{chat_manager_system}"},
    {"role": "user", "content": f"""Analyze step by step and give a detailed correct answer to the Student's question.\n
    Question:\n{topic}\n\nMaterials from the Webinar:\n{message_content}\n\nAnswer:"""}
    ]

  try:
    completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=0.1
    )

    print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
    print('===========================================: \n')
    print('Ответ ChatGPT: ')
    print(completion.choices[0].message.content)
    # return completion.choices[0].message.content
  except:
    print("Модель в настоящее время перегружена. Попробуйте позже.")

def load_txt_file(file_path):
    with open(file_path, 'r') as file_:
        text = file_.read()
    return text

# projects_dir = '/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/'
# name_file = '15_Разбор нейро-сотруд__final_text.txt'
# final_text_dir = projects_dir + name_file

# # Загружаем итоговый текст
# final_text = load_txt_file(final_text_dir)


In [ ]:
#@title  Готовим документ MarkdownHeader по сформированному тексту
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("_#", "Header 1"),
    ("##_", "Header 2"),
    ("###_", "Header 3"),
    ("####_", "Header 4"),
    ("#####_", "Header 5"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(curator.final_text)
md_header_splits

In [ ]:
file_name = 'Курс Нейро-струдники I Занятие 1 I 10.09. 2023_textAll.txt'

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=300
)

# Split
split = text_splitter.split_documents(md_header_splits)
db = FAISS.from_documents(split, OpenAIEmbeddings())
db.save_local(os.path.join(projects_dir, f"db_{file_name[:35]}"))

In [ ]:
split[1]

In [ ]:
topic = """
Каких Нейросотрудников рассмотрим на курсе?
"""

get_chatgpt_answer(topic,  db)